In [12]:
import tabula as tb
import re
import pdfplumber
import pandas as pd
import json

**Specify the PDF File**

In [13]:
PDF_FILE = 'Beverage_CSD (GTC)_2DR - Narrow Short.pdf'

**Define the bounding box to extract the regions:-**
`Measured in number of Points on the page of PDF File`

In [14]:
def create_bounding_box():
    fc = 28.28 # conversion factor
    box = [2,8,20,28] # It is in cm. Defined Intuitively. 
    for i in range(0, len(box)):
        box[i] *= fc
    return box

**JSON Map will be created from here**

In [15]:
def create_json_map(page_col_df,final_df):
    
    #Reset the index of the DataFrame, and use the default one instead. If the DataFrame has a MultiIndex, this method can remove one or more levels.
    page_col_df.reset_index(drop=True, inplace=True)
    final_df.reset_index(drop=True, inplace=True)
    #Concatenate pandas objects along a particular axis with optional set logic along the other axe.{0/’index’, 1/’columns’}, default 0
    final_df_pagewise = pd.concat([page_col_df, final_df], axis=1)
    final_df_pagewise
    
    
    y = final_df_pagewise[final_df_pagewise.apply(lambda r: r.str.contains('DOOR', case=False).any(), axis=1)]
    page_door_record_dict = {}
    door_record_dict = {}
    page_num_prev = ''
    for i in range(len(y)):
        page_num = 'page_'+str(y.iloc[i][y.columns[0]])
        door_num = str(y.iloc[i][y.columns[1]])
        door_num_width = ''
        door_num_depth = ''
        
        
        for j in range(y.shape[1]):
            cell_val = str(y.iloc[i][y.columns[j]])
            if re.search('Width:'+'\s*([a-zA-Z0-9_.-.\']*)',cell_val):
                door_num_width = re.search('Width:'+'\s*([a-zA-Z0-9_.-.\']*)',cell_val).group().split(':')[1]
            if re.search('Depth:'+'\s*([a-zA-Z0-9_.-.\']*)\s*([a-zA-Z0-9_.-.\']*)',cell_val) :
                door_num_depth = re.search('Depth:'+'\s*([a-zA-Z0-9_.-.\']*)\s*([a-zA-Z0-9_.-.\']*)',cell_val).group().split(': ')[1]
        
        if page_num_prev != page_num:
            door_record_dict = {}
            page_num_prev = page_num
        door_record_dict[door_num] =  {"Width":door_num_width, "Depth":door_num_depth}
        page_door_record_dict[page_num] = door_record_dict
    print(page_door_record_dict)
    return page_door_record_dict
    

**PreProcessing of Raw data Extracted from PDF**

In [16]:
def pre_processing(final_df):
    final_df = final_df[final_df[final_df.columns[0]].notna()] # Remove the rows where LOCMCLANE is NaN.
    final_df = final_df.fillna(method='ffill')

    for i in range(len(final_df)):
        loc_mclane = final_df.iloc[i][final_df.columns[0]] #final_df[final_df.columns[0]].values[i] #
        loc_mclane_prev = ''
        # only choose those mclane which contains only integer.
        if loc_mclane.isdecimal():
            loc_mclane_prev = final_df.iloc[i-1][final_df.columns[0]]
            loc_mclane_new = loc_mclane + ' ' + loc_mclane_prev
            final_df.loc[i,final_df.columns[0]] = loc_mclane_new
            final_df_1 = final_df.drop([final_df.index[i]])
            final_df_1 = final_df_1.loc[final_df_1['LOC MCLANE'].str.contains("DOOR|\d+", case=False)]
    return final_df_1

**Add Door Number in the DataFrame Extracted after Cleaning the Data**

In [17]:
def add_door_number(final_df_1):
    
    door_re = re.compile(r'DOOR')
    door = []
    door_num = ''
    door_row = []
    for i in range(len(final_df_1)):
        x = final_df_1.iloc[i][final_df_1.columns[0]]
        if door_re.search(x):
            door_num = x.split()[1]
            door_row.append(i)
        door.append(door_num)
    result = pd.DataFrame(door,columns=['DOOR'])
#     print(door_row)
#     print(door)
    return result,door_row

**Get Final DataFrame**

In [18]:
def get_final_df(result,final_df_1,door_row):
    
    result.reset_index(drop=True, inplace=True)
    final_df_1.reset_index(drop=True, inplace=True)
    final = pd.concat([result, final_df_1], axis=1)
    final = final.drop(final.index[door_row])
    return final

**Extract Raw Data From PDF**

In [19]:
box = create_bounding_box()
required_column = ['LOC MCLANE', 'Desc_A', 'UNIT UPC', 'PACK', 'DESCRIPTION','Size', 'UOM', 'FACINGS']
final_df = []
page_list = []
with pdfplumber.open(PDF_FILE) as pdf:
        pages = pdf.pages
        for i,page in enumerate(pages):
            
            page_number = [i+1] 
            df = []
            
            if page_number[0]>2:
                print("Extracting Data from page:",page_number[0])
                tl = tb.read_pdf(PDF_FILE, pages=page_number,area=[box],output_format="dataframe", stream=True)
                df = tl[0]
                df = df[required_column]              
                final_df.append(df)
        
                page_list =page_list + (page_number*len(df))                

final_df = pd.concat(final_df)
# final_df.to_csv("demo.csv", index=False)
page_col_df = pd.DataFrame(page_list,columns=['Page'])


Extracting Data from page: 3
Extracting Data from page: 3
Extracting Data from page: 4
Extracting Data from page: 4
Extracting Data from page: 5
Extracting Data from page: 5
Extracting Data from page: 6
Extracting Data from page: 6


**Create JSON File**

In [20]:
page_door_record_dict = create_json_map(page_col_df,final_df)
dict_to_json = json.dumps(page_door_record_dict)
print("[INFO]: Successfully created JSON File!!")

json_file = PDF_FILE.split('.')[0] + ".shelves.json"
with open(json_file, "w") as f:
    f.write(dict_to_json)

{'page_3': {'DOOR 1/1': {'Width': '', 'Depth': ''}, 'DOOR 1/2': {'Width': '', 'Depth': ''}, 'DOOR 1/3': {'Width': '', 'Depth': ''}, 'DOOR 1/4': {'Width': '', 'Depth': ''}, 'DOOR 1/5': {'Width': '', 'Depth': ''}}, 'page_4': {'DOOR 1/6': {'Width': '', 'Depth': ''}}, 'page_5': {'DOOR 2/1': {'Width': '', 'Depth': ''}, 'DOOR 2/2': {'Width': '', 'Depth': ''}, 'DOOR 2/3': {'Width': '', 'Depth': ''}, 'DOOR 2/4': {'Width': '', 'Depth': ''}}, 'page_6': {'DOOR 2/5': {'Width': '', 'Depth': ''}, 'DOOR 2/6': {'Width': '', 'Depth': ''}, 'DOOR DISCO': {'Width': '', 'Depth': ''}, 'DOOR NEW': {'Width': '', 'Depth': ''}}}
[INFO]: Successfully created JSON File!!
{'page_3': {'DOOR 1/1': {'Width': '', 'Depth': ''}, 'DOOR 1/2': {'Width': '', 'Depth': ''}, 'DOOR 1/3': {'Width': '', 'Depth': ''}, 'DOOR 1/4': {'Width': '', 'Depth': ''}, 'DOOR 1/5': {'Width': '', 'Depth': ''}}, 'page_4': {'DOOR 1/6': {'Width': '', 'Depth': ''}}, 'page_5': {'DOOR 2/1': {'Width': '', 'Depth': ''}, 'DOOR 2/2': {'Width': '', 'Depth

**Get The DataFrame Extracted from the PDF**

In [21]:
final_df_1 = pre_processing(final_df)
result,door_row = add_door_number(final_df_1)
final = get_final_df(result,final_df_1,door_row)

**Export into CSV or TSV**

In [22]:
csv_file = PDF_FILE.split('.')[0] + '.table.csv'
tsv_file = PDF_FILE.split('.')[0] + '.table.tsv'
final.to_csv(tsv_file,sep='\t', index=False)
final.to_csv(csv_file, index=False)